In [161]:
import numpy as np
from scipy.optimize import linprog
import scipy.sparse as spr
import itertools
from scipy.linalg import sqrtm
from jax.scipy.optimize import minimize
import jax.numpy as jnp
import jax
import sys


# Discrete Choice

In [162]:
K = 2
I = 40
Y = 3
sigma = np.sqrt(1 * 1e-0) * 1


lambda_k = np.array([0,1])
# lambda_k = np.ones(K)



np.random.seed(3987)
φ_i_y_0 =  np.random.normal(0,1, size=[I,Y])
# φ_i_y_0 = np.kron( np.ones((I,1)) , np.random.normal(0,1, size= [1,Y]))

φ_i_y_k = np.random.normal(0,1, size=[I,Y,K])

# φ_i_y_k[:,:,0] = np.ones(I)[:,None]  * np.random.normal(0,1,size= Y)[None,:]


# # add correlation
# A = np.random.rand(K, K)
# A_symmetric = (A + A.T) / 2
# A_pd = A_symmetric + K * np.eye(K)
# A_pd_root = sqrtm(A_pd)

# φ_i_y_k = φ_i_y_k @ A_pd_root


eps_iy = sigma * np.random.gumbel(0,1, size=[I,Y])

y_i_hat = np.argmax(φ_i_y_0 + φ_i_y_k @ lambda_k + eps_iy, axis =1)

### Minimax regret

In [163]:
c = np.concatenate((- φ_i_y_k[range(I),y_i_hat,:].sum(0),np.ones(I)))
A =  np.concatenate(( - φ_i_y_k.reshape([I*Y,K]),  np.kron(np.eye(I), np.ones((Y,1)) ) ), axis =1)
b =  φ_i_y_0.flatten()


res = linprog(c, A_ub = - A, b_ub = - b, bounds=(None, None) , method='highs') 
print(res.fun)

34.08165155092372


In [164]:
print(res.x[:K])
print(lambda_k)

[-0.28185308  0.56028957]
[0 1]


In [165]:
y_i_estimate = np.argmax(φ_i_y_0 + φ_i_y_k @ res.x[:K] , axis =1)
val_i = np.max( φ_i_y_0 + φ_i_y_k @ lambda_k, 1 )

print(np.sum(y_i_estimate == y_i_hat))
print(I)

print( np.inner(c, np.concatenate( (lambda_k, val_i)) ))

24
40
36.861965598975026


### MLE by simulation

We solve

$$ \begin{align*}
    \min_{\lambda, u} &  \frac{1}{S} \sum_{is} u_{is}  - \sum_{ik} \phi_{i\hat{B}_i,k} \lambda_k \\
    s.t. & \quad u_{is} \geq  \phi_{iy0} + \sum_k \phi_{iy,k} \lambda_k + \sigma \epsilon_{isy}  \quad \text{for all } isy \in I \times S \times Y
\end{align*}$$


In [166]:
def compute_MLE_simulation(data, S, seed):
    y_i_hat , φ_i_y_0, φ_i_y_k   = data
    np.random.seed(seed)
    eps_i_s_y = sigma * np.random.gumbel(0,1, size=[I,S,Y])

    φ_is_y_0 = ( φ_i_y_0[:,None,:] +  eps_i_s_y).reshape([I * S, Y])
    φ_is_y_k =( np.ones(S)[None,:,None, None] *  φ_i_y_k[:,None,:,:] ).reshape([I * S, Y , K])


    # Objective function coefficients
    c = np.concatenate((-  φ_i_y_k[range(I),y_i_hat,:].sum(0),  np.ones(I*S)/S  ))

    A =  np.concatenate(( - φ_is_y_k.reshape([I*S*Y,K]),  np.kron(np.eye(I * S), np.ones((Y,1)) ) ), axis =1)
    b =  φ_is_y_0.flatten()

    # Solve the problem
    res = linprog(c, A_ub = - A, b_ub = - b, bounds=(None, None) , method='highs') 
    if res.status != 0:
        print('problem failed')
        
    # print(res.status)
    return res.x[:K]
    
    # print(res.fun / S)



S = 80
seed = 0

data = (y_i_hat , φ_i_y_0, φ_i_y_k)
lambda_S = compute_MLE_simulation(data,S, seed)
print(lambda_S)

[-0.07339495  1.02579245]


In [167]:
### MLE by Gradient Descent
φ_iy_k = φ_i_y_k.reshape([I * Y , K])
φ_iy_0 = φ_i_y_0.reshape([I * Y ])

phi_iy_k = np.concatenate( (φ_iy_k ,φ_iy_0[:,None] ), axis = 1)
muhat_iy = np.array([ np.eye(1,Y, y_i_hat[i])[0] for i in range(I)]).flatten()

# Computing MLE:
alpha = 1 / np.linalg.norm(phi_iy_k @ phi_iy_k.T) 
lambda_t = np.zeros(K)
tol,iter = 1e-6, 20000
for i in range(iter):
    expPhi_i_y = np.exp((φ_iy_0 + φ_iy_k @ lambda_t)/sigma).reshape( (I,Y)) 

    mu_iy = (expPhi_i_y / expPhi_i_y.sum(axis =1)[:,None]).flatten()

    grad_k = ((muhat_iy - mu_iy) @ φ_iy_k).flatten() 

    if np.linalg.norm(grad_k) < tol:
        break
    lambda_t = lambda_t + alpha * grad_k

print('Iterations=', i)
print( 'lambda_k=',lambda_t)
print( 'gradient=', grad_k )

Iterations= 287
lambda_k= [-0.05198963  0.99444237]
gradient= [-7.74762938e-08  9.54807727e-07]


### Standard errors

In [168]:
def compute_asymptotic_variance(lambda_t):
    π_i_y_lambda =( np.exp((φ_i_y_0 + φ_i_y_k @ lambda_t)/sigma)/
                     np.exp((φ_i_y_0 + φ_i_y_k @ lambda_t)/sigma).sum(1)[:,None])
    Delta = np.diag(( π_i_y_lambda ).reshape(I*Y))
    

    H_ll = - φ_iy_k.T @ (Delta - Delta @ np.kron(np.eye(I), np.ones([Y,Y]))  @ Delta  )@ φ_iy_k 

    return -np.linalg.inv(H_ll)

Avar_MLE = compute_asymptotic_variance(lambda_t)
print(Avar_MLE)

[[ 0.05617428 -0.00288433]
 [-0.00288433  0.09148735]]


In [169]:
π_i_y_lambda =( np.exp((φ_i_y_0 + φ_i_y_k @ lambda_t)/sigma)/
                     np.exp((φ_i_y_0 + φ_i_y_k @ lambda_t)/sigma).sum(1)[:,None])
B_hat = ((π_i_y_lambda[:,:,None,None] *  φ_i_y_k[:,:,:,None]  * φ_i_y_k[:,:,None,:] ).sum((0,1)) 
        - (π_i_y_lambda[:,:,None,None,None] * π_i_y_lambda[:,None,:,None,None] 
            *  φ_i_y_k[:,:,None,:,None]  * φ_i_y_k[:,None,:,None,:] ).sum((0,1,2)))

Avar_MLE2 = np.linalg.inv(B_hat)

print(Avar_MLE2)

[[ 0.05617428 -0.00288433]
 [-0.00288433  0.09148735]]


In [170]:
### Bootstrap
iter = 0
R = 100
S = 80
lambda_b_k = []
random_seed = 0
for iter in range(R):
    # print(iter)
    np.random.seed(iter)
    bootstrap_sample = np.random.choice(np.arange(I), size = I, replace=True)

    data = y_i_hat[bootstrap_sample] , φ_i_y_0[bootstrap_sample], φ_i_y_k[bootstrap_sample]
    
    lambda_S = compute_MLE_simulation(data,S, random_seed)
    # print(lambda_S)
    lambda_b_k.append(lambda_S)

lambda_b_k = np.array(lambda_b_k)
Avar_bootstrap = np.cov(lambda_b_k.T)
print(Avar_bootstrap)   

[[0.0849942  0.00115362]
 [0.00115362 0.10103281]]


In [171]:
print('MLE:',lambda_t,' s.e.:',Avar_MLE[np.arange(K), np.arange(K)] **(1/2))
print('Simulated:',lambda_b_k.mean(0),' s.e.:',Avar_bootstrap[np.arange(K), np.arange(K)] **(1/2))
print('MLE2:',lambda_t,' s.e.:',Avar_MLE2[np.arange(K), np.arange(K)] **(1/2))
print(lambda_S)

MLE: [-0.05198963  0.99444237]  s.e.: [0.23701115 0.30246876]
Simulated: [-0.01118097  1.0057336 ]  s.e.: [0.29153765 0.31785659]
MLE2: [-0.05198963  0.99444237]  s.e.: [0.23701115 0.30246876]
[0.01322962 0.90008261]


In [172]:
Avar_MLE2

array([[ 0.05617428, -0.00288433],
       [-0.00288433,  0.09148735]])

In [173]:
Avar_MLE2[np.arange(K), np.arange(K)]

array([0.05617428, 0.09148735])

In [174]:
sys.exit("Stopping execution here.") 

SystemExit: Stopping execution here.

# One-to-one Matching

In [75]:
K = 5
# lambda_k = np.array([-1,2,1,-1,2,3,0])
lambda_k = np.ones(K)*10

X = 100
Y = 100

n_x = np.ones(X)
m_y = np.ones(Y)

np.random.seed(0)
# φ_x_y_0 =  np.random.normal(0,1, size=[X,Y])
φ_x_y_0 =  np.random.normal(1,10, size = [X,1]) * np.random.normal(0,1, size = [1,Y])
φ_x_y_k = np.random.normal(1,10, size=[X,Y,K])
print((φ_x_y_k.sum(2).std())**2)


sigma = np.sqrt(5)
eps_x_y = np.random.normal(0,1, size=[X,Y])

497.6279905331009


In [76]:
φ_x_y = φ_x_y_0 + φ_x_y_k @ lambda_k + sigma * eps_x_y
n_x = np.ones(X)
m_y = np.ones(Y)

In [77]:
np.shape(φ_x_y_0 + φ_x_y_k @ lambda_k + sigma * eps_x_y)

(100, 100)

In [78]:
c = - (φ_x_y).flatten()
A = np.concatenate((np.kron(np.eye(X), np.ones(Y)),
                    np.kron(np.ones((1,X)), np.eye(Y))), axis =0 ) 

b = np.concatenate((n_x,m_y))
res = linprog(c, A_eq = A, b_eq = b, bounds=(0,None), method='highs')

In [79]:
p_hat = - res['eqlin']['marginals'].sum()
mu_xy_hat = res.x

In [80]:
np.all((mu_xy_hat == 0 )  + ( mu_xy_hat == 1))

True

Estimation

In [81]:
φ_xy_k = φ_x_y_k.reshape([X*Y,K])

In [82]:
c = np.concatenate( (- (φ_xy_k * mu_xy_hat[:,None]).sum(0) , np.ones(X+Y) ))
A =  np.block( [φ_xy_k ,- np.kron(np.eye(X), np.ones((Y,1))), - np.kron(np.ones((X,1)), np.eye(Y)) ])
b =  - φ_x_y_0.flatten()
res = linprog(c, A_ub = A, b_ub = b, bounds=(None,None), method='highs')

In [83]:
res.x[:K]

array([10.98941262, 10.42288968, 10.52654521, 10.01121293,  9.8012695 ])

In [84]:
lambda_k

array([10., 10., 10., 10., 10.])

# Bundled choice 

$$ \begin{align*}
    \min_{\lambda, u} & \sum_i u_i  - \sum_{ik} \phi_{i\hat{B}_i,k} \lambda_k \\
    s.t. & \quad u_i \geq  \phi_{iB0} + \sum_k \phi_{iB,k} \lambda_k \quad \text{for all } iB \in I \times 2^Y
\end{align*}$$
Since typically $ \phi_{i\emptyset k}=0 $ for each $k \in [0,K]$ (i.e. the outside option is normilized to zero) we have $u_i \geq 0.$ We also **assume** that $\lambda \geq 0 .$

Since 

$$ \begin{align*}
    \min_{(\lambda, u)\geq 0} & \sum_i u_i  - \sum_k \phi_{i\hat{B}_i,k} \lambda_k \\
    s.t. & \quad u_i - \phi_{iB0} - \sum_k \phi_{iB,k} \lambda_k - s_{iB} = 0  \quad \text{for all } iB \in \mathcal{C} 
\end{align*}$$

In standard form we have the problem
$$ \begin{align*}
    \min_{x \geq 0} & \quad c ^\top x\\
    s.t. & \quad  A x = b 
\end{align*}$$
where
*  $b = \phi^{\mathcal{C}}_{0},$
* $c = ( -\hat{\phi}, \bm{1}_{I}, \bm{0}_{|\mathcal{C}|} ) $ with $ \hat{\phi}_k = \phi_{i\hat{B}_i,k}$ 
* $A = \begin{pmatrix} \Phi^{\mathcal{C}} & \delta^{\mathcal{C}+} & \bm{I}_{|\mathcal{C}|} \end{pmatrix}$ where $\delta^{\mathcal{C}+}_{iB, \ell}  = \bm{1}_{i = \ell}$  is the out-incidence matrix of $\mathcal{C}.$


This problem has $K +I+ |\mathcal{C}| $ variables and $ |\mathcal{C}|$ equality constraints.

### Gross-Substitutes

In [85]:
K = 3
lambda_k = np.array([1,1,1])
# lambda_k = np.array([5]*(K-1) + [1])

I = 10
Y = 3
sigma = np.sqrt(1e-0) * 0


### generate exogenous data

np.random.seed(1)
# φ_i_y_0 =  np.random.normal(0,1, size = [I,Y]) 
φ_i_y_0 =  np.random.normal(1,1, size = [I])[:,None] * np.random.normal(0,1, size = [Y])[None, :]
# φ_i_y_k = np.random.normal(1,1, size=[I,Y,K-1])
φ_i_y_k = np.ones(I)[:,None,None] * np.random.normal(1,1, size=[Y,K-1])[None,:,:]
φ_i_y_k[:,:,0] = np.random.normal(1,1, size=[I])[:,None] * np.ones(Y)[None,:]

eps_iy = np.random.normal(0,1, size = [I,Y])


def φ_k(i,B):
    return np.concatenate( (φ_i_y_k[i,B,:].sum(0), [ - np.sum(B)**(2)]))

def Φ_scalar(i,B, lambda_k):
    
    φ_k = np.concatenate( (φ_i_y_k[i,B,:].sum(0), [ - np.sum(B)**(2)]))

    return φ_i_y_0[i,B].sum() + np.dot(φ_k, lambda_k)

In [86]:
### greedy
def greedy(i,lambda_k, p_y = np.zeros(Y)):

    B_k =  np.zeros(Y, dtype=bool)
    val = Φ_scalar(i,B_k, lambda_k)
    k = 0 

    while k < Y:
        y_k = None
        for y in np.where(1-B_k)[0]:
            val_add_y = Φ_scalar(i,B_k+ np.eye(1,Y,y,dtype=bool)[0] , lambda_k)   - p_y[y]
            if val_add_y > val:
                y_k = y
                val = val_add_y

        if y_k is None:
            break
        else:
            B_k[y_k] = 1
        k += 1
    
    return B_k , val

B_hat_i = np.zeros([I,Y], dtype= bool)

φ_i_y_k += sigma * eps_iy[:,:,None]
for i in range(I):
    B_hat_i[i]= greedy(i, lambda_k)[0]
φ_i_y_k -= sigma * eps_iy[:,:,None]

In [87]:
print(np.mean(B_hat_i.sum(1)))
print(np.std(B_hat_i.sum(1)))
print(np.max(np.sum(B_hat_i,1)))
print(np.min(np.sum(B_hat_i,1)))

1.2
0.4
2
1


Estimation

$$ \begin{align*}
    \min_{x \geq 0} & \quad c ^\top x\\
    s.t. & \quad  A x \geq b 
\end{align*}$$
* $c = \begin{pmatrix} -  \phi_{i\hat{B}_i 0}  \\ \bm{1}_I \end{pmatrix}$
* $A = \begin{pmatrix} -\phi & \bm{I}_I \otimes \bm{1}_{2^Y}  \end{pmatrix}$ 
* $ b =  \phi_0$


In [88]:
# ### brute force

# ### Cost

# c = np.ones([K+I])


# phi_hat = 0

# φ_hat_k = np.zeros([I,K])
# for i in range(I):
#     φ_hat_k[i] = φ_k(i,B_hat_i[i])

# phi_hat =  φ_hat_k.sum(0) 

# c[:K] =  - phi_hat

# ### Constraints
# all_bundles = np.array([np.array(seq) for seq in itertools.product([0, 1], repeat= Y)], dtype = bool)

# phi = np.zeros([I,2 ** Y , K])
# for i in range(I):
#     for id , B in enumerate(all_bundles[1:]):
#         phi[i, id , :] = φ_k(i, B)


# A_all = np.block([- phi.reshape( I * (2**Y ), K) ,  np.kron(np.eye(I), np.ones((2**Y ,1)) ) ])


# phi_0 = np.zeros([I,2 ** Y ] )
# for i in range(I):
#     for id, B in enumerate(all_bundles[1:]):
#         phi_0[i, id] = φ_i_y_0[i,B].sum()

# b_all = phi_0.reshape([I * (2 ** Y )])

# print("starting estimation...")

# res = linprog(c, A_ub = - A_all, b_ub =  - b_all, bounds=(None,None), method='highs')
# # print(res)
# print(res.status)
# print(res.x[:K])
# print(res.fun)

### Column generation:

I start with columns $\mathcal{C} = \{i \hat{B}_i\}_i$

In [89]:
φ_hat_k = np.array([φ_k(i,B_hat_i[i]) for i in range(I)])

### Cost
c = np.ones([K+I])
c[:K] =  - φ_hat_k.sum(0)

### Constant
b_t = np.array([φ_i_y_0[i,B_hat_i[i]].sum() for i in range(I)])

### Matrix
A_t =  np.block([- φ_hat_k,np.eye(I)])

print(np.shape(A_t))

(10, 13)


Algorithm:

In [90]:
iters = 30

res = linprog(c, A_ub = -A_t, b_ub = -b_t, bounds=(None,None), method='highs')

iter = 0
for _ in range(iters):
    B_star_i = []
    val_i = []
    for i in range(I):
        B_star, val = greedy(i,lambda_k= res.x[:K])
        B_star_i.append(B_star)
        val_i.append(val - res.x[  K+ i] ) 

    # print("greedy: done")
    # print(np.max(val_i))
    if np.max(val_i ) < 1e-12:
        print(f"solution found! Iterations: {iter}")
        print(res.fun)
        break
    rows_to_add = np.block([ - np.array([φ_k(i,B_star_i[i]) for i in range(I)]),
                              np.eye(I) ])
    A_t = np.block([[A_t],
                    [ rows_to_add] ])
    b_t = np.concatenate((b_t, np.array([φ_i_y_0[i,B_star_i[i]].sum() for i in range(I)])))
    res = linprog(c, A_ub = - A_t, b_ub = - b_t, bounds=(None,None), method='highs')
    # print("master problem: done")
    # print("##########")
    print(res.x[:K] )
    iter += 1
# print(res.x[:K] )
# print(res.x[:K] / res.x[K-1])

[-1.21414566  0.55723908 -0.72512343]
[-0.12237793  0.35401964 -0.01520942]
[0.15930463 0.30158782 0.16795262]
[1.13866083 0.11929244 0.80477185]
[1.07026697 0.19676473 0.78279077]
solution found! Iterations: 5
18.052630532520713


In [91]:
print((res.x[:K] ).round(5) )
print(lambda_k)
print("#############")
print((res.x[:K] / res.x[K-1]).round(2) )
print(lambda_k / lambda_k[K-1])

[1.07027 0.19676 0.78279]
[1 1 1]
#############
[1.37 0.25 1.  ]
[1. 1. 1.]


### MLE by simulation

In [92]:
S = 2
sigma = 0
eps_i_s_y = sigma * np.random.normal(0,1, size = [I,S,Y]) 
eps_is_y = eps_i_s_y.reshape([I * S, Y])
φ_is_y_0 = ( φ_i_y_0[:,None,:] * np.ones(S)[None,:,None]).reshape([I * S, Y])


### Cost

c = np.ones([K + (I * S)]) / S
 

φ_hat_k = np.array( [φ_k(i,B_hat_i[i]) for i in range(I)] )

c[:K] =  - φ_hat_k.sum(0)

### Constant
b_t =  np.array([ φ_is_y_0[i_s, B_hat_i[i_s // I , :]].sum() 
                        + (eps_is_y[i_s,  B_hat_i[i_s // I,:]]).sum()  
                         for i_s in range(I * S)])

# columns_i = (np.ones(S)[None,:,None]  *  columns_i[:,None,: ]).reshape([I * S , Y])

### Matrix
φ_is_B_hat_k = np.kron( φ_hat_k, np.ones((S,1)) )

A_t =  np.block([- φ_is_B_hat_k, np.eye(I * S)])

print(np.shape(c))
print(np.shape(A_t))
print(np.shape(b_t))

(23,)
(20, 23)
(20,)


In [93]:
res = linprog(c, A_ub = - A_t, b_ub = - b_t, bounds=(None,None), method='highs')
res.x

array([ 0.        ,  0.        ,  0.        ,  3.83707619,  3.83707619,
        0.56765403,  0.56765403,  0.68986383,  0.68986383, -0.106688  ,
       -0.106688  ,  2.7274273 ,  2.7274273 , -1.90299006, -1.90299006,
        4.01321107,  4.01321107,  0.34914129,  0.34914129,  1.92857753,
        1.92857753,  1.09750153,  1.09750153])

In [94]:
iters = 1

res = linprog(c, A_ub = - A_t, b_ub = - b_t, bounds=(None,None), method='highs')

iter = 0
for _ in range(iters):
    B_star_is = []
    val_is = []
    for i_s in range(I * S):
        B_star, val = greedy(i_s % I, lambda_k= res.x[:K], p_y = - eps_is_y[i_s])
        B_star_is.append(B_star)
        val_is.append(val - res.x[K + i_s] ) 

    # print("greedy: done")
    print(np.max(val_is))
    if np.max( val_is ) < 1e-12:
        print(f"solution found! Iterations: {iter}")
        print(res.fun)
        break

    rows_to_add = np.block([ -  np.array([φ_k(i_s // I ,B_star_is[i_s]) for i_s in range(I * S)]) ,
                              np.eye(I * S) ])
    A_t = np.block([[A_t],
                    [rows_to_add] ])

    b_t = np.concatenate((b_t, np.array([ (φ_is_y_0[i_s, B_star_is[i_s]] 
                                           + eps_is_y[i_s, B_star_is[i_s]]).sum(0) 
                                           for i_s in range(I * S)])) )

    res = linprog(c, A_ub = - A_t, b_ub = - b_t, bounds=(None,None), method='highs')
    # print("master problem: done")
    # print("##########")
    # print(res.x[:K] )
    iter += 1
print(res.x[:K] )
# print(res.x[:K] / res.x[K-1] )

5.7400662448390385
[-0. -0. -0.]


In [95]:
np.shape(A_t)

(40, 23)

# One-to-many Matching

In [96]:
# K = 5
# # lambda_k = np.array([-1,2,1,-1,2,3,0])
# lambda_k = np.ones(K)

# X = 100
# Y = 100

# n_x = np.ones(X)
# m_y = np.ones(Y)

# np.random.seed(0)
# # φ_x_y_0 =  np.random.normal(0,1, size=[X,Y])
# φ_x_y_0 =  np.random.normal(0,1, size = [X,1]) * np.random.normal(0,1, size = [1,Y])
# φ_x_y_k = np.random.normal(0,1, size=[X,Y,K])
# print((φ_x_y_k.sum(2).std())**2)


# sigma = np.sqrt(1 * 1e-1)
# eps_x_y = np.random.normal(0,1, size=[X,Y])